# Optimise Prompts

 Guide: [Let the LLM Write the Prompts: An Intro to DSPy in Compound AI Pipelines](https://www.youtube.com/watch?v=I9ZtkgYZnOw)
 
Location 18:19 in video.  

In [2]:
import sys
import os
import dspy 
from common.my_settings import MySettings  
from common.utils import md
from common.llm_client_factory import LlmClientFactory
from dspy_utils.dspy_helpers import md_dspy

settings = MySettings().get()

In [ ]:
lm = dspy.LM(
    'gpt-4.1', 
    model_type='chat', 
    cache=False, 
    api_key=settings.OPENAI_API_KEY
)

dspy.configure(lm=lm)

In [4]:
# Create classes

import dspy 
from pydantic import BaseModel
from typing import Literal

class Place(BaseModel):
    address: str = dspy.InputField()
    name: str = dspy.InputField()
    
class PlaceMatcher(dspy.Signature):
    """Verify that the text is based on the provided context."""
    
    # Inputs
    place_one: Place = dspy.InputField()
    place_two: Place = dspy.InputField()
    
    # Outputs
    is_match: bool = dspy.OutputField(desc="Do the two places refer to the same place?")
    match_confidence: Literal["low", "medium", "high"] = dspy.OutputField()
    rationale: str = dspy.OutputField(desc="Rationale for the decision")
    
matcher = dspy.Predict(PlaceMatcher)
result = matcher(place_one=Place(address="123 Main St, Springfield", name="Springfield Library"),
                 place_two=Place(address="123 Main St, Springfield", name="Springfield Public Library"))

# md(result)

In [5]:
from dspy import Example

trainset = [
    Example(
        place_one={"address": "123 Main St, Springfield", "name": "Springfield Library"},
        place_two={"address": "123 Main St, Springfield", "name": "Springfield Public Library"}
        ,is_match=True
    ).with_inputs("place_one", "place_two"),

    Example(
        place_one={"address": "456 Oak Ave, Shelbyville", "name": "Shelbyville Museum"},
        place_two={"address": "456 Oak Avenue, Shelbyville", "name": "Shelbyville History Museum"},
        is_match=True
    ).with_inputs("place_one", "place_two"),

    Example(
        place_one={"address": "789 Pine Rd, Capital City", "name": "Capital City Zoo"},
        place_two={"address": "101 Maple St, Capital City", "name": "Capital City Aquarium"},
        is_match=False
    ).with_inputs("place_one", "place_two"),

    Example(
        place_one={"address": "202 Birch Blvd, Ogdenville", "name": "Ogdenville Coffee House"},
        place_two={"address": "202 Birch Blvd, Ogdenville", "name": "Ogdenville Café"},
        is_match=True
    ).with_inputs("place_one", "place_two")
    ,

    Example(
        place_one={"address": "303 Cedar Ln, North Haverbrook", "name": "North Haverbrook Gym"},
        place_two={"address": "404 Elm St, North Haverbrook", "name": "North Haverbrook Fitness Center"},
        is_match=False
    ).with_inputs("place_one", "place_two"),
]


In [7]:
prompt_lm = dspy.LM('gpt-3.5-turbo', model_type='chat', cache=False, api_key=os.getenv("OPENAI_API_KEY"))
md(prompt_lm('What model are you and what size are you?'))

# #databricks/databricks-gemma-3-12b
# # mixtral-8x7b-v0-1


['I am a language model AI and I do not have a physical form or size. I exist solely in the digital realm to assist with answering questions and providing information.']

In the context of DSPy and the code you're working with:

```python
tp = dspy.MIPROv2(metric=validate_match, auto="light", prompt_model=prompt_lm, task_model=lm)
```

### 🔹 `prompt_model`
This is typically the **intent model**—the one that will be used in production (PRD) to **generate responses** based on the prompts that DSPy helps optimize. It's the model that will be deployed to handle real user inputs.

- It’s used during training to generate candidate completions.
- It’s the model DSPy is trying to **fine-tune prompts for**.
- It’s usually **smaller or faster**, suitable for real-time inference.

### 🔹 `task_model`
This is the **evaluation model**—used during training to **judge the quality** of the outputs from `prompt_model`.

- It’s not necessarily used in production.
- It helps DSPy decide which prompts are best by scoring or validating them.
- It’s often **larger or more accurate**, used offline for optimization.

### 🔹 `metric=validate_match`
This is your **custom evaluation function**. It defines what “success” looks like for a prediction. DSPy uses this function to determine whether a generated output is correct or useful.

So in summary:

| Component       | Role in DSPy | Role in PRD |
|----------------|--------------|-------------|
| `prompt_model` | Generates candidate outputs during training | **Used in production to serve prompts** |
| `task_model`   | Evaluates candidate outputs during training | Not used in production |
| `metric`       | Defines correctness for training | Not used in production |

Would you like help setting up a pipeline where the optimized prompts are exported for production use with `prompt_model`?

In [8]:
import random

def validate_match(example, pred, trace=None):
    md("**Validate()**", "", "**Example**: ", example, "**Prediction**: ", pred)
    #return example.is_match == pred.is_match  # The models are able to guess this every time so it always gets 100% score and thus doesn't optimise the prompt. Need a harder example.
    return random.choice([True, False])  # randomise the response to force DSPY to try different prompts. Make it erratic on purpose.

from dspy.teleprompt import *

# prompt_lm = common.LlmClientFactory(settings).get_dspy_lm_chat_connecting_to_internal_databricks_model('databricks/databricks-gemma-3-12b')

# mixtral-8x7b-v0-1
#prompt_lm = common.LlmClientFactory(settings).get_dspy_lm_chat_connecting_to_internal_databricks_model('')
#
#md(prompt_lm('hello there who are you?'))

# prompt_model: generates prompts/prompt candidates. Usually smaller model. This is the one used in PRD that the prompts will get called against.
# task_model: AI Judge, evaluates outputs. Usually larger model. May be used internally by DSPy to help refine or rank those outputs.
# validate_match(): Scores outputs of prompt_model
tp = dspy.MIPROv2(metric=validate_match, auto="light", prompt_model=prompt_lm, task_model=lm)
optimized_matcher = tp.compile(matcher, trainset=trainset, requires_permission_to_run=False)

2025/08/20 18:19:00 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: False
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 4

2025/08/20 18:19:00 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/08/20 18:19:00 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/08/20 18:19:00 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


  0%|          | 0/1 [00:00<?, ?it/s]

**Validate()**

**Example**: Example({'place_one': {'address': '123 Main St, Springfield', 'name': 'Springfield Library'}, 'place_two': {'address': '123 Main St, Springfield', 'name': 'Springfield Public Library'}, 'is_match': True}) (input_keys={'place_one', 'place_two'})**Prediction**: Prediction(
    is_match=True,
    match_confidence='high',
    rationale='Both entries share the exact same address ("123 Main St, Springfield"), and their names are highly similar, with the only difference being the inclusion of "Public" in the second name. It is common for institutions like libraries to be referred to both with and without the word "Public" in their names. Given the identical address and the strong similarity in names, it is highly likely these refer to the same place.'
)

100%|██████████| 1/1 [00:02<00:00,  2.07s/it]


...

**Validate()**

**Example**: Example({'place_one': {'address': '303 Cedar Ln, North Haverbrook', 'name': 'North Haverbrook Gym'}, 'place_two': {'address': '404 Elm St, North Haverbrook', 'name': 'North Haverbrook Fitness Center'}, 'is_match': False}) (input_keys={'place_one', 'place_two'})**Prediction**: Prediction(
    is_match=False,
    match_confidence='high',
    rationale='The two places have different addresses ("303 Cedar Ln" vs. "404 Elm St") and different names ("North Haverbrook Gym" vs. "North Haverbrook Fitness Center"). While both are fitness-related businesses in the same town, the differences in both name and address strongly indicate they are separate entities.'
)

Average Metric: 1.00 / 3 (33.3%):  75%|███████▌  | 3/4 [00:02<00:00,  1.32it/s]

**Validate()**

**Example**: Example({'place_one': {'address': '202 Birch Blvd, Ogdenville', 'name': 'Ogdenville Coffee House'}, 'place_two': {'address': '202 Birch Blvd, Ogdenville', 'name': 'Ogdenville Café'}, 'is_match': True}) (input_keys={'place_one', 'place_two'})**Prediction**: Prediction(
    is_match=True,
    match_confidence='high',
    rationale='Both entries share the exact same address ("202 Birch Blvd, Ogdenville"), which strongly suggests they refer to the same physical location. The names "Ogdenville Coffee House" and "Ogdenville Café" are very similar and both describe a coffee-serving establishment, with "Café" and "Coffee House" being near-synonyms. The combination of identical address and similar business type/name makes it highly likely these are the same place, possibly with a slight variation in naming or branding.'
)

Average Metric: 2.00 / 4 (50.0%): 100%|██████████| 4/4 [00:02<00:00,  1.46it/s]

2025/08/20 18:19:58 INFO dspy.evaluate.evaluate: Average Metric: 2 / 4 (50.0%)
2025/08/20 18:19:58 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 50.0 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 0'].
2025/08/20 18:19:58 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [50.0, 50.0, 25.0, 0.0, 50.0, 25.0, 25.0, 25.0, 50.0, 50.0, 50.0]
2025/08/20 18:19:58 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 50.0
2025/08/20 18:19:58 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2025/08/20 18:19:58 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 50.0!


In [ ]:
from pprint import pprint

pprint(optimized_matcher)

In [ ]:
# print(optimized_matcher)
print(optimized_matcher.signature)